## Dependencies

In [1]:
import gc
import re
import warnings
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Conv1D, SpatialDropout1D, Bidirectional, Flatten
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
encoder_path = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
model_path = '/kaggle/input/69-googleq-a-train-use-glove-lstmavg-uncased/model.h5'
tokenizer_path = '/kaggle/input/69-googleq-a-train-use-glove-lstmavg-uncased/tokenizer.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

for feature in text_features:
    test[feature + '_uncased'] = test[feature]

for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())

# Model parameters

In [5]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
MAX_FEATURES = 10000
TITLE_MAX_LEN = 30
BODY_MAX_LEN = 150
ANSWER_MAX_LEN = 150

## Test set

In [6]:
tokenizer = load(tokenizer_path)

# Test features
X_test_title = test['question_title']
X_test_body = test['question_body']
X_test_answer = test['answer']
X_test_title_seq = test['question_title_uncased']
X_test_body_seq = test['question_body_uncased']
X_test_answer_seq = test['answer_uncased']

# Tokenize the sentences
X_test_title_seq = tokenizer.texts_to_sequences(X_test_title_seq)
X_test_body_seq = tokenizer.texts_to_sequences(X_test_body_seq)
X_test_answer_seq = tokenizer.texts_to_sequences(X_test_answer_seq)

# Pad the sentences
X_test_title_seq = pad_sequences(X_test_title_seq, maxlen=TITLE_MAX_LEN)
X_test_body_seq = pad_sequences(X_test_body_seq, maxlen=BODY_MAX_LEN)
X_test_answer_seq = pad_sequences(X_test_answer_seq, maxlen=ANSWER_MAX_LEN)

X_test = [X_test_title, X_test_body, X_test_answer, X_test_title_seq, X_test_body_seq, X_test_answer_seq]

# Model

In [7]:
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

def encoder_block(input_layer):
    encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
    return encoder

## Load embedding

In [8]:
glove_embedding_matrix, glove_unknown_words = build_matrix(tokenizer.word_index, encoder_path, MAX_FEATURES)
print('n unknown words (GloVe): ', len(glove_unknown_words))

n unknown words (GloVe):  685


In [9]:
input_title = Input(shape=(1,), dtype=tf.string, name='input_title')
encoded_title = encoder_block(input_title)

# Body
input_body = Input(shape=(1,), dtype=tf.string, name='input_body')
encoded_body = encoder_block(input_body)

# Answer
input_answer = Input(shape=(1,), dtype=tf.string, name='input_answer')
encoded_answer = encoder_block(input_answer)

# Title sequence
input_title_seq = Input(shape=(TITLE_MAX_LEN,), dtype=tf.float32, name='input_title_seq')
title = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_title_seq)
title = LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(title)
title_out = GlobalAveragePooling1D()(title)

# Body sequence
input_body_seq = Input(shape=(BODY_MAX_LEN,), dtype=tf.float32, name='input_body_seq')
body = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_body_seq)
body = LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(body)
body_out = GlobalAveragePooling1D()(body)

# Answer sequence
input_answer_seq = Input(shape=(ANSWER_MAX_LEN,), dtype=tf.float32, name='input_answer_seq')
answer = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_answer_seq)
answer = LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(answer)
answer_out = GlobalAveragePooling1D()(answer)


# Output
x = Concatenate()([encoded_title, encoded_body, encoded_answer, title_out, body_out, answer_out])
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_initializer='lecun_normal')(x)
x = Dropout(0.5)(x)
output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='lecun_normal', name='output')(x)
model = Model(inputs=[input_title, input_body, input_answer, input_title_seq, input_body_seq, input_answer_seq], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title_seq (InputLayer)    [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_body_seq (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_answer_seq (InputLayer)   [(None, 150)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 300)      17424600    input_title_seq[0][0]            
______________________________________________________________________________________________

# Make predictions

In [10]:
Y_test = model.predict(X_test)

In [11]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.870153,0.548484,0.188571,0.518095,0.703243,0.724198,0.630396,0.576224,0.507314,...,0.872164,0.877332,0.583685,0.929585,0.937327,0.778884,0.086684,0.077954,0.826399,0.899843
1,46,0.888222,0.554076,0.002218,0.715912,0.859781,0.931292,0.565962,0.468493,0.186273,...,0.753806,0.916997,0.607285,0.957321,0.959766,0.813739,0.883960,0.115570,0.081457,0.849230
2,70,0.923216,0.742489,0.006362,0.786436,0.967091,0.967176,0.612289,0.520699,0.221988,...,0.909929,0.896527,0.575541,0.956939,0.955488,0.815432,0.080720,0.071056,0.667247,0.897056
3,132,0.849787,0.460133,0.005574,0.740881,0.760515,0.922323,0.546028,0.443290,0.118249,...,0.680772,0.946349,0.682503,0.969380,0.982598,0.887992,0.826318,0.141119,0.720449,0.895008
4,200,0.943818,0.561115,0.008357,0.870779,0.842235,0.947623,0.644804,0.605731,0.270571,...,0.787239,0.955615,0.670678,0.983648,0.982964,0.888330,0.286583,0.172687,0.769125,0.931267


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.874131,0.573801,0.033206,0.700997,0.798256,0.851049,0.569477,0.484671,0.225278,...,0.773134,0.922924,0.645198,0.957235,0.967428,0.850367,0.548845,0.147715,0.500276,0.896518
std,2812.670060,0.052963,0.127226,0.062795,0.122762,0.102920,0.133628,0.048129,0.083281,0.141555,...,0.091855,0.034007,0.037989,0.022431,0.018928,0.044806,0.317771,0.081996,0.271137,0.026845
min,39.000000,0.686453,0.309148,0.000657,0.285249,0.397445,0.160348,0.477009,0.349174,0.015453,...,0.560113,0.765597,0.540065,0.839469,0.859533,0.680891,0.002214,0.006156,0.027483,0.784145
25%,2572.000000,0.840490,0.470004,0.003812,0.627058,0.744662,0.816238,0.532384,0.423641,0.116503,...,0.702011,0.902388,0.616843,0.947252,0.958605,0.820608,0.257746,0.089729,0.261573,0.878395
50%,5093.000000,0.877494,0.569795,0.008878,0.714600,0.808483,0.897918,0.559924,0.462077,0.188220,...,0.781438,0.928824,0.646158,0.961627,0.971245,0.854997,0.653485,0.138133,0.491149,0.897047
75%,7482.000000,0.912267,0.680500,0.032281,0.792079,0.868584,0.934565,0.600852,0.529642,0.312348,...,0.851700,0.947852,0.672058,0.972786,0.980468,0.881984,0.839500,0.189639,0.738356,0.916714
max,9640.000000,0.980496,0.902467,0.618290,0.944014,0.997056,0.985802,0.714343,0.758362,0.693635,...,0.955093,0.983253,0.747727,0.991744,0.995512,0.955500,0.980614,0.662019,0.979462,0.954034
